In [64]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df = pd.read_csv('cars.csv')

In [69]:
df

,name,economy (mpg),cylinders,displacement (cc),power (hp),weight (lb),0-60 mph (s),year
0,AMC Ambassador Brougham,13.0,8,360.0,175.0,3821,11.0,73
1,AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70
2,AMC Ambassador SST,17.0,8,304.0,150.0,3672,11.5,72
3,AMC Concord DL 6,20.2,6,232.0,90.0,3265,18.2,79
4,AMC Concord DL,18.1,6,258.0,120.0,3410,15.1,78
...,...,...,...,...,...,...,...,...
401,Volvo 145E (Wagon),18.0,4,121.0,112.0,2933,14.5,72
402,Volvo 244DL,22.0,4,121.0,98.0,2945,14.5,75
403,Volvo 245,20.0,4,130.0,102.0,3150,15.7,76
404,Volvo 264GL,17.0,6,163.0,125.0,3140,13.6,78


In [74]:
df = df.drop(df.columns[0], axis=1)
df=df.fillna(df.mean())

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression as linreg
from sklearn.model_selection import cross_val_score as cvs



In [83]:
X

,economy (mpg),cylinders,displacement (cc),weight (lb),0-60 mph (s),year
0,13.0,8,360.0,3821,11.0,73
1,15.0,8,390.0,3850,8.5,70
2,17.0,8,304.0,3672,11.5,72
3,20.2,6,232.0,3265,18.2,79
4,18.1,6,258.0,3410,15.1,78
...,...,...,...,...,...,...
401,18.0,4,121.0,2933,14.5,72
402,22.0,4,121.0,2945,14.5,75
403,20.0,4,130.0,3150,15.7,76
404,17.0,6,163.0,3140,13.6,78


In [76]:
X = df.drop(['power (hp)'], axis=1)
# y = df['dep'].values.reshape(-1,1)
target = df['power (hp)']

In [78]:
msk = np.random.rand(len(df)) < 0.8
X_train = X[msk]
X_test = X[~msk]

y_train = target[msk]
y_test = target[~msk]


In [79]:
np.any(np.isnan(df))

False

In [88]:
X_test

,economy (mpg),cylinders,displacement (cc),weight (lb),0-60 mph (s),year
1,15.0,8,390.0,3850,8.5,70
3,20.2,6,232.0,3265,18.2,79
8,18.0,6,232.0,2789,15.0,73
9,19.0,6,232.0,2634,13.0,71
13,18.0,6,199.0,2774,15.5,70
...,...,...,...,...,...,...
377,29.8,4,89.0,1845,15.3,80
389,29.0,4,97.0,1940,14.5,77
395,41.5,4,98.0,2144,14.7,80
400,19.0,4,121.0,2868,15.5,73


In [89]:
lin_reg = linreg()

MSEs = cvs(lin_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

mean_MSE = np.mean(MSEs)

print(mean_MSE)

-189.95646043357416


In [81]:
model = Ridge(alpha=1e-2).fit(X_train, y_train)
model.score(X_test, y_test)

0.8844063369175778

In [90]:
from sklearn.inspection import permutation_importance as pfi

r = pfi(model, X_test, y_test, n_repeats=30, random_state=0)


for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X_test.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

weight (lb)0.418 +/- 0.066
displacement (cc)0.286 +/- 0.048
0-60 mph (s)0.217 +/- 0.044
cylinders0.049 +/- 0.012
